# Node.js 교과서 4장 
# < http 모듈로 서버 만들기 >
## 4.1 요청과 응답 이해하기

In [8]:
%%writefile code/4/server1.js

const http = require('http');

http.createServer((req, res) => {
    res.writeHead(200, {'Content-Type': 'text/html; charset=utf-8'});
    res.write('<h1>Hello Node!</h1>');
    res.write('<p>Hello server ^^</p>');
    res.end('<p>Bye ~</p>');
})
    .listen(8080, ()=>{ //서버 연결
        console.log('8080번 포트에서 서버 대기 중입니다.');
    });

Overwriting code/4/server1.js


server1.js 실행 후 http://localhost:8080 (또는 http://127.0.0.1:8080) 접속 결과
![http](img/http.png)

In [9]:
%%writefile code/4/server1-1.js

const http = require('http');

const server = http.createServer((req, res) => {
    res.writeHead(200, {'Content-Type': 'text/html; charset=utf-8'});
    res.write('<h1>Hello Node!</h1>');
    res.write('<p>Hello server ^^</p>');
    res.end('<p>Bye ~</p>');
});
server.listen(8080);
//listen 콜백 대신 이벤트 리스너 붙이기

server.on('listening', ()=>{ //서버 연결
    console.log('8080번 포트에서 서버 대기 중입니다.');
});
server.on('error', console.log);
    

Overwriting code/4/server1-1.js


In [11]:
%%writefile code/4/server1-2.js
//한번에 여러서버 실행
const http = require('http');

http.createServer((req, res) => {
    res.writeHead(200, {'Content-Type': 'text/html; charset=utf-8'});
    res.write('<h1>Hello Node!</h1>');
    res.write('<p>Hello server ^^</p>');
    res.end('<p>Bye ~</p>');
})
    .listen(8080, ()=>{ //서버 연결1
        console.log('8080번 포트에서 서버 대기 중입니다.');
    });
    
http.createServer((req, res) => {
    res.writeHead(200, {'Content-Type': 'text/html; charset=utf-8'});
    res.write('<h1>Hello Node!</h1>');
    res.write('<p>Hello server ^^</p>');
    res.end('<p>Bye ~</p>');
})
    .listen(8081, ()=>{ //서버 연결2
        console.log('8081번 포트에서 서버 대기 중입니다.');
    });

Overwriting code/4/server1-2.js


### 파일 내용 읽어서 띄우기

In [13]:
%%writefile code/4/server2.html
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8" />
    <title>Node.js 웹 서버</title>
</head>
<body>
    <h1>Node.js 웹 서버</h1>
    <p>만들 준비되셨나요?</p>
</body>
</html>

Writing code/4/server2.html


In [17]:
%%writefile code/4/server2.js

const http = require('http');
const fs = require('fs').promises;

http.createServer(async (req, res) => {
  try {
    const data = await fs.readFile('./server2.html');//data에 html 파일 내용 저장
    res.writeHead(200, { 'Content-Type': 'text/html; charset=utf-8' });
    res.end(data);//data 전달
  } catch (error) {
    console.error(error);
    res.writeHead(500, { 'Content-Type': 'text/plain; charset=utf-8' });
    res.end(error.message);
  }
})
  .listen(8081, () => {
    console.log('8081번 포트에서 서버 대기 중입니다!');
  });

Overwriting code/4/server2.js


## 4.2 REST와 라우팅 사용하기
- REpresentational State Transfer(REST):  
서버의 자원을 정의하고 자원에 대한 주소를 지정하는 방법(일종의 약속)

- [ HTTP 요청 메서드 ]
 - GET: 서버 자원을 가져오고자 할 때 사용(요청 본문에 데이터를 넣지 않음, 쿼스트링 사용)
 - POST: 서버에 자원을 새로 등록하고자 할 때 사용 (요청 본문에 데이터를 넣어 보냄)
 - PUT: 서버의 자원을 요청에 들어있는 자원으로 치환하고자 할 때 사용 (요청 본문에 데이터를 넣어 보냄)
 - PATCH: 서버의 자원의 일부만 수정하고자 할 때 사용 (요청 본문에 데이터를 넣어 보냄)
 - DELETE: 서버의 자원을 삭제하고자 할 때 사용
 - OPTIONS: 요청 하기 전 통신 옵션을 설명하기 위해 사용

In [18]:
%%writefile code/4/restFront.css
a { color: blue; text-decoration: none; }

Writing code/4/restFront.css


In [19]:
%%writefile code/4/restFront.html

<!DOCTYPE html>
<html lang="ko">
<head>
  <meta charset="utf-8" />
  <title>RESTful SERVER</title>
  <link rel="stylesheet" href="./restFront.css" />
</head>
<body>
<nav>
  <a href="/">Home</a>
  <a href="/about">About</a>
</nav>
<div>
  <form id="form">
    <input type="text" id="username">
    <button type="submit">등록</button>
  </form>
</div>
<div id="list"></div>
<script src="https://unpkg.com/axios/dist/axios.min.js"></script>
<script src="./restFront.js"></script>
</body>
</html>


Writing code/4/restFront.html


In [24]:
%%writefile code/4/about.html

<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8" />
  <title>RESTful SERVER</title>
  <link rel="stylesheet" href="./restFront.css" />
</head>
<body>
<nav>
  <a href="/">Home</a>
  <a href="/about">About</a>
</nav>
<div>
  <h2>소개 페이지입니다.</h2>
  <p>사용자 이름을 등록하세요!</p>
</div>
</body>
</html>


Writing code/4/about.html


In [23]:
%%writefile code/4/restFront.js

async function getUser() { // 로딩 시 사용자 가져오는 함수
  try {
    const res = await axios.get('/users');
    const users = res.data;
    const list = document.getElementById('list');
    list.innerHTML = '';
    // 사용자마다 반복적으로 화면 표시 및 이벤트 연결
    Object.keys(users).map(function (key) {
      const userDiv = document.createElement('div');
      const span = document.createElement('span');
      span.textContent = users[key];
      const edit = document.createElement('button');
      edit.textContent = '수정';
      edit.addEventListener('click', async () => { // 수정 버튼 클릭
        const name = prompt('바꿀 이름을 입력하세요');
        if (!name) {
          return alert('이름을 반드시 입력하셔야 합니다');
        }
        try {
          await axios.put('/user/' + key, { name });
          getUser();
        } catch (err) {
          console.error(err);
        }
      });
      const remove = document.createElement('button');
      remove.textContent = '삭제';
      remove.addEventListener('click', async () => { // 삭제 버튼 클릭
        try {
          await axios.delete('/user/' + key);
          getUser();
        } catch (err) {
          console.error(err);
        }
      });
      userDiv.appendChild(span);
      userDiv.appendChild(edit);
      userDiv.appendChild(remove);
      list.appendChild(userDiv);
      console.log(res.data);
    });
  } catch (err) {
    console.error(err);
  }
}

window.onload = getUser; // 화면 로딩 시 getUser 호출
// 폼 제출(submit) 시 실행
document.getElementById('form').addEventListener('submit', async (e) => {
  e.preventDefault();
  const name = e.target.username.value;
  if (!name) {
    return alert('이름을 입력하세요');
  }
  try {
    await axios.post('/user', { name });
    getUser();
  } catch (err) {
    console.error(err);
  }
  e.target.username.value = '';
});


Overwriting code/4/restFront.js


In [29]:
%%writefile code/4/restServer.js

const http = require('http');
const fs = require('fs').promises;

const users = {}; // 데이터 저장용

http.createServer(async (req, res) => {
  try {
    if (req.method === 'GET') {// GET요청
      if (req.url === '/') { //http://localhost:8082/ 페이지 요청했을 때
        const data = await fs.readFile('./restFront.html');
        res.writeHead(200, { 'Content-Type': 'text/html; charset=utf-8' });
        return res.end(data);
      } else if (req.url === '/about') { //http://localhost:8082/about 페이지 요청했을 때
        const data = await fs.readFile('./about.html');
        res.writeHead(200, { 'Content-Type': 'text/html; charset=utf-8' });
        return res.end(data);
      } else if (req.url === '/users') {
        res.writeHead(201, { 'Content-Type': 'application/json; charset=utf-8' }); //json형식으로 전송
        return res.end(JSON.stringify(users));
      }
      // /도 /about도 /users도 아니면
      try {
        const data = await fs.readFile(`.${req.url}`);
        return res.end(data);
      } catch (err) {
        // 주소에 해당하는 라우트를 못 찾았다는 404 Not Found error 발생
      }
    } else if (req.method === 'POST') {
      if (req.url === '/user') {
        let body = '';
        // 요청의 body를 stream 형식으로 받음
        req.on('data', (data) => {
          body += data;
        });
        // 요청의 body를 다 받은 후 실행됨
        return req.on('end', () => {
          console.log('POST 본문(Body):', body);
          const { name } = JSON.parse(body);
          const id = Date.now();
          users[id] = name;
          res.writeHead(201, { 'Content-Type': 'text/plain; charset=utf-8' });
          res.end('ok');
        });
      }
    } else if (req.method === 'PUT') {
      if (req.url.startsWith('/user/')) {
        const key = req.url.split('/')[2];
        let body = '';
        req.on('data', (data) => {
          body += data;
        });
        return req.on('end', () => {
          console.log('PUT 본문(Body):', body);
          users[key] = JSON.parse(body).name;
          res.writeHead(200, { 'Content-Type': 'text/plain; charset=utf-8' });
          return res.end('ok');
        });
      }
    } else if (req.method === 'DELETE') {
      if (req.url.startsWith('/user/')) {
        const key = req.url.split('/')[2];
        delete users[key];
        res.writeHead(200, { 'Content-Type': 'text/plain; charset=utf-8' });
        return res.end('ok');
      }
    }
    res.writeHead(404);
    return res.end('NOT FOUND');
  } catch (err) {
    console.error(err);
    res.writeHead(500, { 'Content-Type': 'text/plain; charset=utf-8' });
    res.end(err.message);
  }
})
  .listen(8082, () => {
    console.log('8082번 포트에서 서버 대기 중입니다');
  });


Overwriting code/4/restServer.js


HTTP 응답코드: https://developer.mozilla.org/ko/docs/Web/HTTP/Status

## 4.3 쿠키와 세션 이해하기
요청을 보냈을 때 서버에서는 누가 요청을 보냈는지 모름...  
로그인구현에 쿠키와 세션 필요  
- 쿠키: 키-값의 쌍, 매 요청마다 서버에 동봉해서 쿠키를 보내면 서버는 쿠키를 읽어서 누구인지 파악

In [32]:
%%writefile code/4/cookie.js

const http = require('http');

http.createServer((req, res) => {
  console.log(req.url, req.headers.cookie);
  res.writeHead(200, { 'Set-Cookie': 'mycookie=test' }); //쿠키: mycookie=test
  res.end('Hello Cookie');
})
  .listen(8083, () => {
    console.log('8083번 포트에서 서버 대기 중입니다!');
  });

Overwriting code/4/cookie.js


/ undefined  
/favicon.ico <a style="color:red;">mycookie=test </a>

/(첫번째 요쳥): 쿠키 정보 없음
/favicon.ico(두번째 요청): favicon(웹사이트 탭에 보이는 이미지)를 웹브라우저가 요청한 것, 쿠키 정보를 포함하고 있음

![cookie.png](img/cookie.png)

### 쿠키를 이용해서 사용자를 식별하기

In [41]:
%%writefile code/4/cookie2.html

<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8" />
    <title>쿠키&세션 이해하기</title>
</head>
<body>
<form action="/login">
    <input id="name" name="name" placeholder="이름을 입력하세요" />
    <button id="login">로그인</button>
</form>
</body>
</html>

Overwriting code/4/cookie2.html


In [44]:
%%writefile code/4/cookie2.js

// 쿠키에 한글값이 들어가면 문제생김...
// 인코딩을 해주는 parseCookies
const http = require('http');
const fs = require('fs').promises;
const url = require('url');
const qs = require('querystring');

const parseCookies = (cookie = '') =>
  cookie
    .split(';')
    .map(v => v.split('='))
    .reduce((acc, [k, v]) => {
      acc[k.trim()] = decodeURIComponent(v);
      return acc;
    }, {});

http.createServer(async (req, res) => {
  const cookies = parseCookies(req.headers.cookie);
  // 주소가 /login으로 시작하는 경우
  if (req.url.startsWith('/login')) {
    const { query } = url.parse(req.url);
    const { name } = qs.parse(query);
    const expires = new Date();
    // 쿠키 유효 시간을 현재시간 + 5분으로 설정
    expires.setMinutes(expires.getMinutes() + 5);
    res.writeHead(302, {
      Location: '/',
      'Set-Cookie': `name=${encodeURIComponent(name)}; Expires=${expires.toGMTString()}; HttpOnly; Path=/`,
    });
    res.end();
  // name이라는 쿠키가 있는 경우
  } else if (cookies.name) {
    res.writeHead(200, { 'Content-Type': 'text/plain; charset=utf-8' });
    res.end(`${cookies.name}님 안녕하세요`);
  } else {
    try {
      const data = await fs.readFile('./cookie2.html');
      res.writeHead(200, { 'Content-Type': 'text/html; charset=utf-8' });
      res.end(data);
    } catch (err) {
      res.writeHead(500, { 'Content-Type': 'text/plain; charset=utf-8' });
      res.end(err.message);
    }
  }
})
  .listen(8084, () => {
    console.log('8084번 포트에서 서버 대기 중입니다!');
  });


Overwriting code/4/cookie2.js


~~~js
res.writeHead(302, { //302: 리다이렉션  
      Location: '/',  
      'Set-Cookie': `name=${encodeURIComponent(name)}; //인코딩  
                     Expires=${expires.toGMTString()}; //쿠키 만료기간 설정  
                     HttpOnly; //자바스크립트로 접근비허용(보안문제)  
                     Path=/`,  // path 하위항목에서 쿠키 유효함  
    });  
~~~

**쿠키옵션**
- 쿠키명 = 쿠키값
- Expires = 날짜 :쿠키 만료기간, 기본값은 클라이언트가 종료될 때 까지
- Max-age = 초 :Expires와 비슷하지만 날짜 대신 초를 입력할 수 있음
- Domain = 도메인명 :쿠키가 전송될 도메인 특정, 기본값은 현재 도메인
- Path = URL :쿠키가 전송될 URL 특정, 기본값 '/'(모든 URL에서 쿠키 전송)
- Secure: https인 경우에만 쿠키가 전송됨
- HttpOnly: 자바스크립트에서 쿠키에 접근할 수 없음(쿠키 조작 방지)

### 세션 사용하기
(쿠키보다 안전한 방법)

In [45]:
%%writefile code/4/session.js

const http = require('http');
const fs = require('fs').promises;
const url = require('url');
const qs = require('querystring');

const parseCookies = (cookie = '') =>
  cookie
    .split(';')
    .map(v => v.split('='))
    .reduce((acc, [k, v]) => {
      acc[k.trim()] = decodeURIComponent(v);
      return acc;
    }, {});

const session = {}; //데이터저장용

http.createServer(async (req, res) => {
  const cookies = parseCookies(req.headers.cookie);
  if (req.url.startsWith('/login')) {
    const { query } = url.parse(req.url);
    const { name } = qs.parse(query);
    const expires = new Date();
    expires.setMinutes(expires.getMinutes() + 5);
    const uniqueInt = Date.now(); //uniqueInt를 key로 사용
    session[uniqueInt] = {
      name,
      expires, 
    };
    res.writeHead(302, {
      Location: '/',
      'Set-Cookie': `session=${uniqueInt}; Expires=${expires.toGMTString()}; HttpOnly; Path=/`,
    });
    res.end();
  // 세션쿠키가 존재하고, 만료 기간이 지나지 않았다면
  } else if (cookies.session && session[cookies.session].expires > new Date()) {
    res.writeHead(200, { 'Content-Type': 'text/plain; charset=utf-8' });
    res.end(`${session[cookies.session].name}님 안녕하세요`);
  } else {
    try {
      const data = await fs.readFile('./cookie2.html');
      res.writeHead(200, { 'Content-Type': 'text/html; charset=utf-8' });
      res.end(data);
    } catch (err) {
      res.writeHead(500, { 'Content-Type': 'text/plain; charset=utf-8' });
      res.end(err.message);
    }
  }
})
  .listen(8085, () => {
    console.log('8085번 포트에서 서버 대기 중입니다!');
  });


Writing code/4/session.js


Set-Cookie: session=1601128738068; Expires=Sat, 26 Sep 2020 14:03:58 GMT; HttpOnly; Path=/  
세션쿠키로 key만 넘겨짐, 서버에서 key를 이용해 처리

## 4.4 https와 http2
http는 보안상 문제 발생 가능  
보안을 위해서 https 사용하는 것이 좋다  
**https.createServer(cert, key, ca, callback())**  
<a style='color:skyblue'>https는 인증서가 있어야 사용가능하므로... 실습은 할수 없음</a>

In [47]:
%%writefile code/4/https.js

const https = require('https'); //https모듈
const fs = require('fs');

https.createServer({
  cert: fs.readFileSync('도메인 인증서 경로'), //인증서가 있다면
  key: fs.readFileSync('도메인 비밀키 경로'),  //입력할수 있겠지만
  ca: [
    fs.readFileSync('상위 인증서 경로'), //인증서를 받는게
    fs.readFileSync('상위 인증서 경로'), //어렵다고 한다
  ],
}, (req, res) => {
  res.writeHead(200, { 'Content-Type': 'text/html; charset=utf-8' });
  res.write('<h1>Hello Node!</h1>');
  res.end('<p>Hello Server!</p>');
})
  .listen(443, () => {
    console.log('443번 포트에서 서버 대기 중입니다!');
  });

Overwriting code/4/https.js


**http2는 속도를 향상시킨 버전(인증서필요)**

In [48]:
%%writefile code/4/http2.js

const http2 = require('http2'); //http2모듈
const fs = require('fs');

http2.createSecureServer({
  cert: fs.readFileSync('도메인 인증서 경로'),
  key: fs.readFileSync('도메인 비밀키 경로'),
  ca: [
    fs.readFileSync('상위 인증서 경로'),
    fs.readFileSync('상위 인증서 경로'),
  ],
}, (req, res) => {
  res.writeHead(200, { 'Content-Type': 'text/html; charset=utf-8' });
  res.write('<h1>Hello Node!</h1>');
  res.end('<p>Hello Server!</p>');
})
  .listen(443, () => {
    console.log('443번 포트에서 서버 대기 중입니다!');
  });

Writing code/4/http2.js


## 4.5 cluster
싱글 프로세스로 동작하는 노드가 CPU 코어를 모두 사용할 수 있게 해주는 모듈  
- 장점: 코어 하나당 노드 프로세스 하나 배정 가능(성능이 개선됨)
- 단점: 자원 공유 불가능

In [55]:
%%writefile code/4/cluster.js

const cluster = require('cluster');
const http = require('http');
const numCPUs = require('os').cpus().length; //cpu 개수 구하기

if (cluster.isMaster) { //마스터 스레드에서
  console.log(`마스터 프로세스 아이디: ${process.pid}`);
  // CPU 개수만큼 워커를 생산
  for (let i = 0; i < numCPUs; i += 1) {
    cluster.fork();
  }
  // 워커가 종료되었을 때
  cluster.on('exit', (worker, code, signal) => {
    console.log(`${worker.process.pid}번 워커가 종료되었습니다.`);
    console.log('code', code, 'signal', signal);
    //cluster.fork(); //프로세스를 다시 생성
  });
} else {
  // 워커들이 포트에서 대기
  http.createServer((req, res) => {
    res.writeHead(200, { 'Content-Type': 'text/html; charset=utf-8' });
    res.write('<h1>Hello Node!</h1>');
    res.end('<p>Hello Cluster!</p>');
      
    setTimeout(() => { // 워커 존재를 확인하기 위해 1초마다 강제 종료
      process.exit(1);
    }, 1000);
   
  }).listen(8086); //서버 여러개가 8086포트를 함께 사용

  console.log(`${process.pid}번 워커 실행`);
}

Overwriting code/4/cluster.js


워커 4개가 생성되었다가 종료되었다.
~~~
C:\Users\maild\node-js-study\code\4>node cluster.js
마스터 프로세스 아이디: 16984
5804번 워커 실행
21076번 워커 실행
22260번 워커 실행
23268번 워커 실행
23268번 워커가 종료되었습니다.
code 1 signal null
22260번 워커가 종료되었습니다.
code 1 signal null
21076번 워커가 종료되었습니다.
code 1 signal null
5804번 워커가 종료되었습니다.
code 1 signal null
~~~